# Bài toán: IEEE-CIS Fraud Detection

Đây là notebook về Data analysis. Modeling notebook ở đây: https://www.kaggle.com/tunnguynnhanh/ml-assignment-modeling

Bài toán dự đoán một giao dịch có phải là gian lận hay không (fraud transaction detection). Vesta là đơn vị cung cấp dữ liệu cho bài toán.

Dữ liệu của gồm 2 file: **transaction** và **identity** được nối với nhau bằng cột **TransactionID**. Nghĩa là sẽ có tổng cộng 4 file, 2 file cho tập train, và 2 file cho tập test. Mô tả chi tiết 2 file ở bên dưới:

1. File transaction

* TransactionDT: Thời điểm thực hiện giao dịch tính từ 1 thời điểm cố định (khoảng thời gian delta, chứ không phải là một datetime)
* TransactionAMT: số tiền thanh toán giao dịch đơn vị USD
* ProductCD [categorical]: Mã của sản phẩm trong giao dịch, sản phẩm không nhất thiết là một đồ vật, mà có thể là một dịch vụ
* card1 - card6 [categorical]: thông tin thẻ thanh toán, ví dụ như loại thẻ, ngân hàng, quốc gia,...
* addr1 - addr2 [categorical]: địa chỉ khu vực thanh toán - quốc gia thanh toán
* dist1 - dist2: thông tin về khoảng cách
* P_emaildomain [categorical]: email domain của người mua
* R_emaildomain [categorical]: email domain của người nhận
* C1 - C14: thông tin về số lượng, như là bao nhiêu địa chỉ tìm thấy liên quan đến thẻ thanh toán, v...v... Ý nghĩa thực được giấu đi
* D1 - D15: thông tin khoảng thời gian (delta), ví dụ như khoảng thời gian tính từ giao dịch gần nhất
* M1 - M9 [categorical]: thông tin đúng sai (boolean), thể hiện kết nối giữa 2 thông tin, ví dụ như tên trên thẻ thanh toán với địa chỉ
* Vxxx: những features được thiết kế bởi Vesta

2. File identity

* TransactionID: ID của giao dịch đó
* DeviceType [categorical]: loại thiết bị dùng để giao dịch
* DeviceInfo [categorical]: thông tin chi tiết hơn về thiết bị đã sử dụng
* id1 - id38 [categorical + numerical]: thông tin về mạng (network connection), trình duyệt sử dụng (browser) (id 12-38 là categorical feature)

Mô tả rõ hơn về data: https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import datetime
import re
import warnings
from IPython.display import Image

# Load dữ liệu từ bộ data của cuộc thi

In [ ]:
data_path = "../input/ieee-fraud-detection/"

In [ ]:
# Transaction CSVs
train_transaction = pd.read_csv(data_path + "train_transaction.csv")
test_transaction = pd.read_csv(data_path + "test_transaction.csv")

# Identity CSVs
train_identity = pd.read_csv(data_path + "train_identity.csv")
test_identity = pd.read_csv(data_path + "test_identity.csv")

# Sample Submissions
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

# Định nghĩa một vài hàm utils để sử dụng cho phần phân tích dữ liệu

In [ ]:
def df_shape(df):
    print("The shape of data is: rows --> %s, columns --> %s" % (df.shape[0], df.shape[1]))
    print("")
    
def df_first_look(df):
    """
    Nhận vào một pandas dataframe và thể hiện những thông tin cơ bản: shape, 5 dòng đầu, column name lists
    """
    try:
        print()
        df_shape(df)
        print("First 5 rows of dataframe:\n--------------------------\n", df.head())
        print("")
        print(
            "Columns list:\n---------------------------------\n",
            df.columns.values,
        )
        print("")
        print(
            "Looking NaN values and datatypes of columns in the dataframe:\n--------------------------------------------\n"
        )
        print(df.info())
        print("")

    except Exception as e:
        print("Error at df_first_look function: ", str(e))

In [ ]:
def df_missing(df):
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing(%)'] = df.isnull().sum().values / train.shape[0]*100  
    summary['Uniques'] = df.nunique().values
    return summary

# Xem các dữ liệu lần đầu tiên để có đánh giá sơ bộ

In [ ]:
df_first_look(train_transaction)

In [ ]:
df_first_look(train_identity)

In [ ]:
df_first_look(test_transaction)

In [ ]:
df_first_look(test_identity)

==> Nhận xét: Số lượng features là khá lớn, ví dụ như feature V ở bảng transaction, hơn nữa, cột V và các cột ở bảng Identity có nhiều giá trị missing. Ta sẽ phải xử lý giá trị missing này, cũng như nếu cần thì giảm số lượng features xuống.

# Ghép 2 bảng transaction và identity thành một bảng train duy nhất, làm tương tự với file test

In [ ]:
train_identity['TransactionID'].unique().shape

Ta đếm số lượng các transaction ở vừa có mặt ở bảng `transaction` và `identity` để lựa chọn cách ghép 2 bảng này với nhau. Nếu ở bảng `transaction` có nhiều `TransactionID` nằm trong `identity` hơn thì sẽ merge theo bảng `transaction` và ngược lại

In [ ]:
print(np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique())))
print(np.sum(train_identity['TransactionID'].isin(train_transaction['TransactionID'].unique())))

Ở tập train, có 24.4% TransactionIDs trong bảng `train tracsaction` (144233 / 590540) có liên kết với `train_identity`. Còn khi đếm số lượng `TransactionID` ở trong bảng `identity` thì có đủ ở trong bảng `transaction`. Vậy ta sẽ merge như bên dưới, giống như merge bảng trong sql. Điều này đồng nghĩa với việc sẽ tạo ra vấn đề là có lượng lớn missing data ở các cột thuộc bảng `identity` mà ta cần giải quyết.

In [ ]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [ ]:
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [ ]:
df_first_look(train)

# Xử lí dữ liệu missing (null/ NaN) 

Sử dụng hàm utils để biết thêm thông tin missing

In [ ]:
train_missing = df_missing(train)
test_missing = df_missing(test)

In [ ]:
train_missing

In [ ]:
train_missing.sort_values(by=['Missing(%)'], ascending=False)[train_missing['Missing(%)'] > 0].shape[0]

=> Có 414 cột features trên tổng số 434 cột có giá trị missing trong tập `train`

In [ ]:
test_missing

In [ ]:
test_missing.sort_values(by=['Missing(%)'], ascending=False)[test_missing['Missing(%)'] > 0].shape[0]

=> Có 385 cột features trên tổng số 433 cột có giá trị missing trong tập `test`

In [ ]:
col_70missing = train_missing[train_missing['Missing(%)'] >70]['Name'].to_list()
print('Features có hơn 70% missing value trong tập train là: ')
print(col_70missing)

=> Những features có tỉ lệ missing values lớn là feature V,D,id. 

==> Sau khi nhìn qua thông tin missing. Ta thấy rằng cả tập `train` và `test` đều có số lượng lớn features có giá trị missing. Có khá nhiều features có tỉ lệ missing lớn hơn 70% như trên. Tuy nhiên, em chưa xử lý nhiều các giá trị missing này vì có thể nó lại có ảnh hưởng tốt đến model. Ở đây, em chỉ xóa đi những features có tỉ lệ missing > 99%, với tỉ lệ missing lớn như vậy thường sẽ không có giá trị gì trong quá trình training model.

In [ ]:
col_99missing = train_missing[train_missing['Missing(%)'] >= 99]['Name'].to_list()
print('Features có hơn 99% missing value trong tập train là: ')
print(col_99missing)

In [ ]:
train.drop(columns=col_99missing,inplace=True)
# test.drop(columns=col_99missing, inplace=True)

==> Khá may mắn, trong quá trình xóa các features có tỉ lệ missing > 99% ở tập `test` thì gặp lỗi. Nhìn lại thông tin ở hàm df_first look ở trên thì tập test, các cột `id_01` bị thay dấu `_` bằng `-`. Giờ thay tên cột và áp dụng drop col99missing

In [ ]:
# Đổi tên cột các cột id cho giống với train data
test.rename({'id-01':'id_01','id-02':'id_02','id-03':'id_03','id-04':'id_04','id-05':'id_05','id-06':'id_06','id-07':'id_07','id-08':'id_08','id-09':'id_09','id-10':'id_10','id-11':'id_11','id-12':'id_12','id-13':'id_13','id-14':'id_14','id-15':'id_15','id-16':'id_16','id-17':'id_17','id-18':'id_18','id-19':'id_19','id-20':'id_20','id-21':'id_21','id-22':'id_22','id-23':'id_23','id-24':'id_24','id-25':'id_25','id-26':'id_26','id-27':'id_27','id-28':'id_28','id-29':'id_29','id-30':'id_30','id-31':'id_31', 'id-32':'id_32', 'id-33':'id_33', 'id-34':'id_34', 'id-35':'id_35', 'id-36':'id_36', 'id-37':'id_37', 'id-38':'id_38'}, axis=1, inplace=True)

In [ ]:
test.drop(columns=col_99missing, inplace=True)

==> Ở đây em cũng kiểm tra xem có feature nào chỉ có 1 giá trị unique ở tập test thì cũng xóa đi luôn

In [ ]:
one_unique_col = [i for i in train.columns if train[i].nunique() == 1]
one_unique_col_test = [i for i in test.columns if test[i].nunique() == 1]
print("Có %d features ở tập train có 1 giá trị unique" % len(one_unique_col))
print("Có %d features ở tập test có 1 giá trị unique" % len(one_unique_col_test))

In [ ]:
# Có 1 feature có 1 giá trị unique ở tập test nên em tìm feature đó và xóa đi 
print(one_unique_col_test)

In [ ]:
train.drop(columns='V107',inplace=True)
test.drop(columns='V107',inplace=True)

# Đánh giá và xử lý dữ liệu

## Numerical Features

**TransactionDT**

Là thời gian giao dịch tính từ một mốc thời gian cụ thể (không phải là 1 giá trị timestamp). Đọc các discussion thì có thể biết được đơn vị của TransactionDT tính bằng giây.

In [ ]:
TransactionDT_train_min = train['TransactionDT'].min()
TransactionDT_train_max = train['TransactionDT'].max()
TransactionDT_test_min = test['TransactionDT'].min()
TransactionDT_test_max = test['TransactionDT'].max()

print("Train min: {}".format(TransactionDT_train_min))
print("Train max: {}".format(TransactionDT_train_max))
print("Test min: {}".format(TransactionDT_test_min))
print("Test max: {}".format(TransactionDT_test_max))

==> Nhìn kết quả ở trên ta thấy có các mốc thời gian khác nhau giữa tập `train` và tập `test`. Có thể plot ra để thấy rõ hơn.

In [ ]:
train_transaction["TransactionDT"].plot(
    kind="hist",
    figsize=(15, 5),
    label="train",
    bins=60,
    title="Train vs Test TransactionDT Distribution",
)
test_transaction["TransactionDT"].plot(kind="hist", label="test", bins=50)
plt.legend()

In [ ]:
train_span = (TransactionDT_train_max - TransactionDT_train_min)/(3600*24)
test_span = (TransactionDT_test_max - TransactionDT_test_min)/(3600*24)
total_span = (TransactionDT_test_max - TransactionDT_train_min)/(3600*24)
gap_span = (TransactionDT_test_min - TransactionDT_train_max)/(3600*24)
print('Time span của tập train là {:.2f} ngày'.format(train_span))
print('Time span của tập test là {:.2f} ngày'.format(test_span))
print('Time span của cả tập dữ liệu là {:.2f} ngày'.format(total_span))
print('Khoảng cách giữa tập train và tập test là {:.2f} ngày'.format(gap_span))

==>
**Vấn đề**: Dữ liệu `TransactionDT` là các mốc thời gian khác nhau, nếu giữ nguyên thì sẽ ảnh hưởng đến mô hình dạng cây.

**Giải quyết**: Vì giá trị `TransactionDT` là giây nên ta có thể đổi sang các feature về "day", "hour"

**Nhận xét**: Em không chuyển thành "month" và "year" vì dữ liệu chỉ được lấy trong 1 năm, 2 dữ liệu đó sẽ không có giá trị nhiều


In [ ]:
train['day'] = ((train['TransactionDT']//(3600*24)-1)%7)+1
train['hour'] = ((train['TransactionDT']//3600)%24)+1

==> Sau khi có 2 features mới, thì ta cần xem tương quan của 2 features đó với thông tin quan trọng là tỉ lệ fraud transaction.

In [ ]:
train_day = train.groupby('isFraud')['day'].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values('day')
plt.figure(figsize=(10,6))
barplot = sns.barplot(x="day", y="percentage", hue="isFraud", data=train_day, palette = 'pastel')
plt.legend()
plt.ylabel('percentage of transaction frequency')
plt.xlabel('Day (not neccessary corresponding to the exact week days)')
plt.title('Percentage of fraudulent and legit transactions frequency in Train')
for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')
plt.show()

==> Nhìn vào biểu đồ, ta thấy tỉ lệ `fraud transaction` ở các ngày (không nhất thiết là các ngày trong tuần) không quá khác biệt. Không có một trend quá rõ ràng nào.

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(train.groupby('hour').mean()['isFraud'], color='r')
ax = plt.gca()
ax2 = ax.twinx()
_ = ax2.hist(train['hour'], alpha=0.3, bins=24)
ax.set_xlabel('hours (not neccesary corresponding to the real hours)')
ax.set_ylabel('Fraction of fraudulent transactions')
ax2.set_ylabel('Number of transactions')
plt.title('Number of transactions (blue hist) versus fraction of fraudulent transactions (red Polyline)')
plt.xticks(np.arange(1, 25, 1))
plt.show()

=> Đường màu đỏ thể hiện tỉ lệ số transaction là fraud, cột màu xanh thể hiện số lượng transactions trong giờ đó. Nhận xét: từ 4 - 12 giờ, tỉ lệ fraud transaction lớn hơn so với khung giờ khác, trong đó thì từ 7 - 10 giờ là cao nhất. Mặt khác, tỉ lệ fraud transaction thấp nhất trong khoảng thời gian từ 14 - 16 giờ. Ta có thể thấy một reverse trend ở đây, vào các thời điểm tổng số transactions thấp thì tỉ lệ fraud transaction lại cao. Mấy tên phạm tội thường thực hiện lúc mọi người đang ngủ hoặc vào thời điểm không ai để ý. 

Dựa vào nhận xét trên thì ta có thể tạo ra thêm một feature nữa `hour` bằng feature khác:
- Từ 7 - 10 giờ là "high warning signal" => Có nguy cơ cao xảy ra fraud
- Từ 14 - 16 giờ là "lowest warning signal" => Ít nguy cơ xảy ra fraud nhất
- Từ 4 - 7 giờ và 10 - 14 giờ là "medium warning signal" => Có nguy cơ xảy ra fraud
- Các mốc còn lại là "low warning signal" => It nguy cơ xảy ra fraud


In [ ]:
def new_hr_feature(hr):
    if hr >= 7 and hr < 10:
        return "highwarningsign"
    if hr >= 14 and hr < 16:
        return "lowestwarningsign"
    if (hr >= 4 and hr < 7) or (hr >= 10 and hr < 14):
        return "mediumwarningsign"
    else:
        return "lowwarningsign"

In [ ]:
train['hour_warning'] = train['hour'].apply(new_hr_feature)

In [ ]:
# Áp dụng các features "day", "hour", "hour_warning" cho tập test
test['day'] = ((test['TransactionDT']//(3600*24)-1)%7)+1
test['hour'] = ((test['TransactionDT']//3600)%24)+1
test['hour_warning'] = test['hour'].apply(new_hr_feature)

**TransactionAmt**

Số tiền thanh toán giao dịch bằng USD.

**Vấn đề**: các numerical feature có thể có outlier (các giá trị đặc biệt lớn, hoặc đặc biệt nhỏ), những giá trị này có thể làm ảnh hưởng đến mô hình

**Giải pháp**: đánh giá qua giá trị `mean`, `min` và `max`. Ta có thể lấy những giá trị đó bằng hàm describe()

In [ ]:
train['TransactionAmt'].describe()

In [ ]:
test['TransactionAmt'].describe()

==> Qua việc describe đơn giản 2 cột dữ liệu ở tập `train` và `test` thì ta thấy giá trị `mean` là tương đối gần rồi. Nhưng giá trị `max` ở tập `train` lại gấp khoảng 3 lần so với tập `test`. Điều này có thể là do có các giá trị outlier (giá trị rất lớn), những giá trị này sẽ ảnh hưởng lớn đến model khi training và cần loại bỏ nó đi. Ta có thể vẽ biểu đồ để kiểm chứng điều này

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
g1 = sns.scatterplot(x="TransactionDT",y="TransactionAmt",hue="isFraud", data=train, alpha=0.8, hue_order=[0,1])
plt.title('TransactionDT vs TransactionAmt by isFraud in Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="TransactionAmt", data=test, alpha=0.8, hue_order=[0,1])
plt.title('TransactionDT vs TransactionAmt by isFraud in Test')
plt.show()

Ta thấy ở tập `train` có một điểm rất lớn, chính là điểm lớn nhất có giá trị hơn 30000, trong khi tập `test` chỉ ở khoảng giá trị 10000. 1 cách đơn giản là tìm các row ở tập `train` có giá trị TransactionAmt > 10270 (giá trị max ở tập test) rồi xóa chúng đi.

In [ ]:
train[train.TransactionAmt>10270]

Ta thấy chỉ có 2 rows có giá trị TransactionAmt > 10270. Ta sẽ xóa chúng đi luôn

In [ ]:
train.drop(train[train.TransactionAmt>10270].index, axis=0, inplace=True)

**C1-C14**

Continuous variables.

Meaning: các feature số lượng, ví dụ như là số địa chỉ tìm thấy liên quan đến thẻ thanh toán, ... Ý nghĩa thật được giấu đi bởi bên cung cấp data.  

Ta liệt kê ra các cột features C để tiện làm việc. Và sử dụng kĩ thuật giống như ở phần `TransactionAmt`, ta kiểm tra cột nào có outliers rồi xóa chúng đi

In [ ]:
C_cols = ["C{}".format(i) for i in range(1, 15)]
train[C_cols].describe()

In [ ]:
test[C_cols].describe()

==> Như phần tích phần `TransactionDT`, ta sẽ nhìn vào giá trị mean và max. Nếu giá trị mean có sự chênh lệch lớn hoặc giá trị mean tương đương nhưng max của train lớn hơn của test nhiều thì nghĩa là đang có giá trị outliers. Nhận thấy chỉ có cột `C5` và `C9` không có bất thường. Các cột khác đều phải xử lý

In [ ]:
train.drop(train[train.C1>3000].index, axis=0, inplace=True)
train.drop(train[train.C2>3000].index, axis=0, inplace=True)
train.drop(train[train.C7>1400].index, axis=0, inplace=True)
train.drop(train[train.C4>1600].index, axis=0, inplace=True)
train.drop(train[train.C6>1700].index, axis=0, inplace=True)
train.drop(train[train.C7>1600].index, axis=0, inplace=True)
train.drop(train[train.C8>1000].index, axis=0, inplace=True)
train.drop(train[train.C10>1000].index, axis=0, inplace=True)
train.drop(train[train.C11>2300].index, axis=0, inplace=True)
train.drop(train[train.C12>2300].index, axis=0, inplace=True)
train.drop(train[train.C13>1600].index, axis=0, inplace=True)
train.drop(train[train.C14>800].index, axis=0, inplace=True)

In [ ]:
train[C_cols].describe()

In [ ]:
test[C_cols].describe()

**Dist1 và Dist2**

Features về khoảng cách, giống như những giá trị numerical khác, em cũng tìm các điểm outliers và loại bỏ những điểm đó đi.

In [ ]:
train[['dist1', 'dist2']].describe()

In [ ]:
test[['dist1', 'dist2']].describe()

==> Giống như nhận xét ở trên, có thể có giá trị outliers do giá trị mean có sự chênh lệch đáng kể. Nhưng do không rõ ràng nên ta sẽ thể bằng biểu đồ để đánh giá

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
g1 = sns.scatterplot(x="TransactionDT",y="dist1",hue="isFraud", data=train, alpha=0.8, hue_order=[0,1])
plt.title('Dist1 vs TransationDT in Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="dist1", data=test, alpha=0.8, hue_order=[0,1])
plt.title('Dist1 vs TransationDT in Test')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
g1 = sns.scatterplot(x="TransactionDT",y="dist2",hue="isFraud", data=train, alpha=0.8, hue_order=[0,1])
plt.title('Dist2 vs TransationDT in Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="dist2", data=test, alpha=0.8, hue_order=[0,1])
plt.title('Dist2 vs TransationDT in Test')
plt.show()

==> Từ biểu đồ ta thấy được các giá trị dist1 lớn hơn 8000 là outliers, dist2 lớn hơn 10000 là outliers, ta có thể xóa chúng đi

In [ ]:
train.drop(train[train.dist1>8000].index, axis=0, inplace=True)
train.drop(train[train.dist2>10000].index, axis=0, inplace=True)

**D1 - D15**

D là features thể hiện time delta. Tuy không có thông tin chính xác về dữ liệu này. Nhưng qua đọc các discussion. Em biết được rằng `D9` cũng là thông tin về `day` giống như đã làm ở trên. Do đã có dữ liệu `day` rồi nên em để nguyên feature `D9`

Cũng giống với các features khác, em thử describe để tìm các điểm outliers.

In [ ]:
D_cols = ["D{}".format(i) for i in range(1, 16)]
train[D_cols].describe()

In [ ]:
test[D_cols].describe()

Các giá trị không có gì quá đặc biệt. Các điểm trên không có gì đặc biệt cả, một điểm cần lưu ý đã trình bày ở trên là D có tỉ lệ missing khá cao.

Do D cũng là dữ liệu timedelta giống `TransactionDT`, ta có thể vẽ biểu đồ thể hiện mối quan hệ giữa 2 feature này.

In [ ]:
plt.figure(figsize=(20,50))
var = ['D' + str(i) for i in range(1,16)]
i = 1
for col in var:
    plt.subplot(8,4,i)
    sns.scatterplot(x="TransactionDT",y=col,hue="isFraud",data=train[~train[col].isnull()])
    plt.title('Train '+col)
    i += 1
    plt.subplot(8,4,i)
    sns.scatterplot(x="TransactionDT",y=col,data=test[~test[col].isnull()])
    plt.title('Test '+col)
    i += 1
plt.show()

==> Nhìn vào biểu đồ, ở đa số các cột, giá trị của feature D sẽ tăng khi TransactionDT tăng, điều này cũng khá hợp lí với mô tả của dữ liệu. Feature D này em không có xử lí gì cả

**V1 - V339**

Dữ liệu V cũng là dữ liệu có số lượng nhiều nhất trong tập dữ liệu, nhưng ý nghĩa của tập V thì lại không được nói rõ. Em cũng đã đọc các discussion thì tập V này không có nhiều thông tin. Ở phần thử nghiệm model, khi em in ra độ quan trọng của các features thì đa số feature V đều đứng cuối bảng. Điều này đồng nghĩa với việc có thể phải giảm số chiều của V để model đỡ nặng

In [ ]:
V_cols = ["V{}".format(i) for i in range(1, 340) if i != 107]
train[V_cols].describe()

In [ ]:
test[V_cols].describe()

==> Dữ liệu V em cũng không xử lý gì. Em sẽ để việc giảm chiều dữ liệu ở phần modeling để so sánh xem việc giảm chiều dữ liệu có tác động tốt đến model hay không.

## Categorical features


**Card1 - Card6**

Thông tin thẻ thanh toán, chẳng hạn như loại thẻ, loại thẻ, ngân hàng phát hành, quốc gia, v.v. Ta sẽ đi qua lần lượt từng feature


In [ ]:
# Reference: https://github.com/KaustuvDash/IEEE-Fraud-Detection
def describe(datatrain,datatest,feature):
    d = pd.DataFrame(columns=[feature,'Train','TrainFraud','TrainLegit','Test'])
    d[feature] = ['count','mean','std','min','25%','50%','75%','max','unique','NaN','NaNshare']
    for i in range(0,8):
        d['Train'].iloc[i] = datatrain[feature].describe().iloc[i]
        d['TrainFraud'].iloc[i]=datatrain[datatrain['isFraud']==1][feature].describe().iloc[i]
        d['TrainLegit'].iloc[i]=datatrain[datatrain['isFraud']==0][feature].describe().iloc[i]
        d['Test'].iloc[i]=datatest[feature].describe().iloc[i]
    d['Train'].iloc[8] = len(datatrain[feature].unique())
    d['TrainFraud'].iloc[8]=len(datatrain[datatrain['isFraud']==1][feature].unique())
    d['TrainLegit'].iloc[8]=len(datatrain[datatrain['isFraud']==0][feature].unique())
    d['Test'].iloc[8]=len(datatest[feature].unique())
    d['Train'].iloc[9] = datatrain[feature].isnull().sum()
    d['TrainFraud'].iloc[9] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()
    d['TrainLegit'].iloc[9] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()
    d['Test'].iloc[9]=datatest[feature].isnull().sum()
    d['Train'].iloc[10] = datatrain[feature].isnull().sum()/len(datatrain)
    d['TrainFraud'].iloc[10] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==1])
    d['TrainLegit'].iloc[10] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==0])
    d['Test'].iloc[10]=datatest[feature].isnull().sum()/len(datatest)
    return d

In [ ]:
def plot(feature):
  plt.figure(figsize=(12,6))
  plt.subplot(1,2,1)
  sns.distplot(train[(train['isFraud']==0) & (~train[feature].isnull())][feature])
  sns.distplot(train[(train['isFraud']==1) & (~train[feature].isnull())][feature])
  plt.ylabel('Probability Density')
  plt.legend(['legit','fraud'])
  plt.title('Train')
  plt.suptitle('{} Distribution'.format(feature) , fontsize=12)
  plt.subplot(1,2,2)
  sns.distplot(test[~test[feature].isnull()][feature])
  plt.title('Test')
  plt.show()

In [ ]:
def plot2(feature):
    train[feature].describe()
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    train_card = (train[~train[feature].isnull()].groupby(['isFraud'])[feature].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values(feature))
    sns.barplot(x="{}".format(feature), y="percentage", hue="isFraud", data=train_card, palette = 'pastel')
    plt.title('Train')

Card1

In [ ]:
train['card1'].head()
# plot2('card1')

Card2

In [ ]:
# describe(train, test, 'card2')
plot('card2')

Card3

In [ ]:
# describe(train, test, 'card3')
plot('card3')

Card4

In [ ]:
# describe(train, test, 'card4')
plot('card4')
# plot("card4")
# plt.figure(figsize=(12,6))
# train_card4 = (train[~train['card4'].isnull()].groupby(['isFraud'])['card4'].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values('card4'))
# sns.barplot(x="card4", y="percentage", hue="isFraud", data=train_card4, palette = 'pastel')
# plt.title('Percentage of fraud and legit across card types')
# plt.show()

=> Chủ yếu giao dịch được thực hiện bằng thẻ mastercard hoặc visa. Ta cũng không thấy trend gì đặc biệt ở đây

Card5

In [ ]:
# describe(train, test, 'card5')
plot('card5')

Card6

In [ ]:
train['card6'].describe()
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card6 = (train[~train['card6'].isnull()].groupby(['isFraud'])['card6'].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values('card6'))
sns.barplot(x="card6", y="percentage", hue="isFraud", data=train_card6, palette = 'pastel')
plt.title('Train')

In [ ]:
print('%d transactions bằng charge card' % train[train['card6']=='charge card'].shape[0])
print('%d transactions bằng debit or credit card' % train[train['card6']=='debit or credit'].shape[0])

=> Ta thấy tỉ lệ transaction thực hiện bằng `change card` và `debit or credit` đều rất ít, lần lượt là 15 và 30 giao dịch. Ta sẽ xem ở tập test thì biểu đồ sẽ như thế nào.

In [ ]:
plt.subplot(1,2,2)
test_card6 =test[~test['card6'].isnull()]['card6'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
sns.barplot(x="index", y="percentage", data=test_card6, palette = 'pastel')
plt.xlabel('card6')
plt.title('Test')
plt.suptitle('Percentage of fraud and legit transactions frequency by card types', fontsize=12)
plt.show()

=> Ta thấy ở tập `test` còn không có giao dịch có loại là "debet or credit", "change card" cũng rất ít. Lại thấy đa số giao dịch ở loại "debit", nên ta sẽ chuyển những cột "change card" và "debit or credit" thành "debit" luôn

In [ ]:
def replacetodebit(row):
    if row==np.nan:
        return row
    if row=='debit or credit' or row=='charge card':
        return 'debit'
    else:
        return row

In [ ]:
#áp dụng cho cả 2 tập test và train
train['card6'] = train['card6'].apply(replacetodebit)
test['card6'] = test['card6'].apply(replacetodebit)

### addr1 and addr2
addr1 - Purchaser Region, addr2 - Purchaser Billing Country

In [ ]:
describe(train, test, 'addr1')

In [ ]:
plot('addr1')

In [ ]:
describe(train, test, 'addr2')

In [ ]:
plot('addr2')

==> Dữ liệu không có gì bất thường. Chỉ có một nhận xét nhỏ là đối với feature `addr2`, dữ liệu tập trung chủ yếu ở khoảng 85-87, có thể đây là các tài khoản ở Mỹ vì Vesta - đơn vị cung cấp data cũng ở Mỹ

### P_emaildomain and R_emaildomain ###
P_emaildomain là email của purchaser

R_emaildomain là email của receipent

==> Domain nhiều nhất là gmail, yahoo (những domain khá phổ biến)

In [ ]:
plt.figure(figsize=(20,8))
plt.subplot(1,2,1)
train_P_email = (train[~train['P_emaildomain'].isnull()].groupby(['isFraud'])['P_emaildomain'].value_counts(normalize=True).rename('percentage').mul(100).reset_index())
sns.barplot(x="P_emaildomain", y="percentage", hue="isFraud", data=train_P_email, palette = 'pastel')
plt.xticks(rotation=90)
plt.subplot(1,2,2)
train_R_email = train[~train['R_emaildomain'].isnull()].groupby(['isFraud'])['R_emaildomain'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
sns.barplot(x="R_emaildomain", y="percentage", hue="isFraud", data=train_R_email, palette = 'pastel')
plt.xticks(rotation=90)
plt.suptitle('Percentage of fraud and legit by email domains', fontsize=18)
plt.show()

==> Nhìn chung, những domain có fraud transaction vẫn là những domain phổ biến. Tuy nhiên có rất nhiều domain có số lượng transactions ít, ta nên xếp chúng vào một category là "others". Với `P_emaildomain`, gộp theo tên tổ chức và các domain có dưới 500 transactions thì sẽ được gộp lại là "others". Với `R_emaildomain`, domain có dưới 300 transactions sẽ được gộp lại là "others"

In [ ]:
# Tham khảo: https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt/notebook#Ploting-Transaction-Amount-Values-Distribution
train.loc[train['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
train.loc[train['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo'
train.loc[train['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
train.loc[train.P_emaildomain.isin(train.P_emaildomain.value_counts()[train.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
train.P_emaildomain.fillna("NoInf", inplace=True)

train.loc[train['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

train.loc[train['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo'
train.loc[train['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
train.loc[train.R_emaildomain.isin(train.R_emaildomain.value_counts()[train.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
train.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
# plot sau khi chia lại các domain

plt.figure(figsize=(20,8))
plt.subplot(1,2,1)
train_P_email = (train[~train['P_emaildomain'].isnull()].groupby(['isFraud'])['P_emaildomain'].value_counts(normalize=True).rename('percentage').mul(100).reset_index())
sns.barplot(x="P_emaildomain", y="percentage", hue="isFraud", data=train_P_email, palette = 'pastel')
plt.xticks(rotation=90)
plt.subplot(1,2,2)
train_R_email = train[~train['R_emaildomain'].isnull()].groupby(['isFraud'])['R_emaildomain'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
sns.barplot(x="R_emaildomain", y="percentage", hue="isFraud", data=train_R_email, palette = 'pastel')
plt.xticks(rotation=90)
plt.suptitle('Percentage of fraud and legit by email domains', fontsize=18)
plt.show()

In [ ]:
# áp dụng cho tập test 

test.loc[test['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
test.loc[test['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo'
test.loc[test['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
test.loc[test.P_emaildomain.isin(test.P_emaildomain.value_counts()[test.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
test.P_emaildomain.fillna("NoInf", inplace=True)

test.loc[test['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

test.loc[test['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo'
test.loc[test['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
test.loc[test.R_emaildomain.isin(test.R_emaildomain.value_counts()[test.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
test.R_emaildomain.fillna("NoInf", inplace=True)

**M1 - M9**

Feature boolean

In [ ]:
m_features = list(train_transaction.columns[46:55])
train_transaction[m_features].head()

### ProductCD ###

In [ ]:
plt.figure(figsize=(6,6))
train_ProductCD = (train.groupby(['isFraud'])['ProductCD'].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values('ProductCD'))
sns.barplot(x="ProductCD", y="percentage", hue="isFraud", data=train_ProductCD,palette = 'pastel')
plt.title('Percentage of fraud and legit across ProductCD in Train')
plt.show()

==> Đa số fraud transaction đều thuộc ProductCD là C, W. Tuy nhiên, với type C, thì số lượng non-fraud transaction lại rất ít nên có thể nhận xét là nếu productCD là C thì khả năng cao là fraud transaction

### DeviceType and DeviceInfo ###

In [ ]:
# Xem các giá trị của 2 features này
train[~train.iloc[:, 422:424].isnull().any(axis=1)].iloc[:, 422:424].head()

In [ ]:
# Xem mỗi feature có bao nhiêu giá trị
print(len(train['DeviceType'].value_counts()))
print(len(train['DeviceInfo'].value_counts()))

==> `DeviceType` thì có 2 giá trị là mobile, và desktop. Sẽ check thử xem có gì đặc biệt như là mobile thì hay fraud hơn hay không. DeviceInfo có khá nhiều giá trị, sẽ tìm cách giải quyết

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
train_DeviceType = (train[~train['DeviceType'].isnull()].groupby(['isFraud'])['DeviceType'].value_counts(normalize=True).rename('percentage').mul(100).reset_index().sort_values('DeviceType'))
sns.barplot(x="DeviceType", y="percentage", hue="isFraud", data=train_DeviceType, palette = 'pastel')
plt.title('Train')

plt.subplot(1,2,2)
test_DeviceType =test[~test['DeviceType'].isnull()]['DeviceType'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
sns.barplot(x="index", y="percentage", data=test_DeviceType, palette = 'pastel')
plt.xlabel('DeviceType')
plt.title('Test')

plt.show()

==> Plot ra thì tỉ lệ không có gì đặc biệt, vì tỉ lệ fraud ở cả desktop và mobile đều bằng nhau. Số lượng desktop và mobile cũng k chênh nhau quá nhiều

**id_1 - id_38**

id gồm các thông tin xác định của giao dịch như device rating, ip_domain rating, proxy rating

các cột id ban đầu có 38 cột từ, sau khi xóa 'id_07', 'id_08', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27' do có tỉ lệ missing value >= 99% thì còn 29 cột

In [ ]:
train[~train.iloc[:, 393:422].isnull().any(axis=1)].iloc[:, 393:422].head()

https://www.kaggle.com/c/ieee-fraud-detection/discussion/108575 tham khảo ở đây về kĩ thuật splitting

Ta thấy có 3 cột có thể xử lí. Cột `id_30`, `id_31`, `id_33`. `id_31` cho ta thông tin về browser của transaction, nhưng lại chưa tách biệt version, ta có thể tách version để có 1 feature mới. `id_30` cho biết về hệ điều hành, cũng có version, và ta nên tách ra. `id_33` cho biết thông tin về resolution của cái gì đó, nếu để giá trị như vậy thì số lượng values ctrong cột `id_33` sẽ lớn, ta sẽ tách 2 giá trị resolution ra.

In [ ]:
train['OS_id_30'] = train['id_30'].str.split(' ', expand=True)[0]
train['version_id_30'] = train['id_30'].str.split(' ', expand=True)[1]
test['OS_id_30'] = test['id_30'].str.split(' ', expand=True)[0]
test['version_id_30'] = test['id_30'].str.split(' ', expand=True)[1]

train['browser_id_31'] = train['id_31'].str.split(' ', expand=True)[0].value_counts()
train['version_id_31'] = train['id_31'].str.split(' ', expand=True)[1].value_counts()
test['browser_id_31'] = test['id_31'].str.split(' ', expand=True)[0].value_counts()
test['version_id_31'] = test['id_31'].str.split(' ', expand=True)[1].value_counts()

train['resol_width_id_31'] = train['id_33'].str.split('x', expand=True)[0].value_counts()
train['resol_height_id_31'] = train['id_33'].str.split('x', expand=True)[1].value_counts()
test['resol_width_id_31'] = test['id_33'].str.split('x', expand=True)[0].value_counts()
test['resol_height_id_31'] = test['id_33'].str.split('x', expand=True)[1].value_counts()

Sau khi tách các thông tin đó ra thì xóa các cột cũ đi.

In [ ]:
train.drop(columns=['id_30', 'id_31', 'id_33'], inplace=True)
test.drop(columns=['id_30', 'id_31', 'id_33'], inplace=True)

# Export Data

Trong quá trình thực hiện bài toán, em thường xuyên rơi vào tình trạng tràn ram. Nên em thường export dữ liệu ra để thực hiện phần modeling dễ hơn.

In [ ]:
# train.to_csv("train.csv")

In [ ]:
# test.to_csv("test.csv")